In [1]:
import os
import sys

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(PROJECT_ROOT)

from src.paths import RAW_AUDIO_DIR, PROCESSED_AUDIO_DIR
print("RAW_AUDIO_DIR:", RAW_AUDIO_DIR)
print("PROCESSED_AUDIO_DIR:", PROCESSED_AUDIO_DIR)

RAW_AUDIO_DIR: c:\Users\user\OneDrive\Desktop\portfolio\portfolio_projects\DRI_SWAASA-main\data\raw\audio
PROCESSED_AUDIO_DIR: c:\Users\user\OneDrive\Desktop\portfolio\portfolio_projects\DRI_SWAASA-main\data\processed\audio


In [2]:
import librosa
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis

In [3]:
def extract_audio_features(audio_path, filename, sr=22050, n_mfcc=13):
    y, sr = librosa.load(audio_path, sr=sr)
    features = {}

    features["filename"] = filename
    features["recording_length"] = librosa.get_duration(y=y, sr=sr)

    # RMS
    rms = librosa.feature.rms(y=y)[0]
    features["rms_power"] = np.mean(rms)

    # ZCR
    zcr = librosa.feature.zero_crossing_rate(y=y)[0]
    features["zero_crossing_rate"] = np.mean(zcr)

    # Crest Factor
    peak = np.max(np.abs(y))
    features["crest_factor"] = peak / (np.sqrt(np.mean(y**2)) + 1e-8)

    # MFCCs
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    for i in range(n_mfcc):
        features[f"mfcc_{i+1}_mean"] = np.mean(mfcc[i])
        features[f"mfcc_{i+1}_std"] = np.std(mfcc[i])

    # Spectrum
    spectrum = np.abs(librosa.stft(y))
    freqs = librosa.fft_frequencies(sr=sr)
    psd = np.mean(spectrum**2, axis=1)

    features["spectral_centroid"] = np.mean(
        librosa.feature.spectral_centroid(y=y, sr=sr)
    )
    features["spectral_rolloff"] = np.mean(
        librosa.feature.spectral_rolloff(y=y, sr=sr)
    )
    features["spectral_spread"] = np.mean(
        librosa.feature.spectral_bandwidth(y=y, sr=sr)
    )
    features["spectral_flatness"] = np.mean(
        librosa.feature.spectral_flatness(y=y)
    )

    features["spectral_skewness"] = skew(psd)
    features["spectral_kurtosis"] = kurtosis(psd)
    features["spectral_std"] = np.std(psd)

    slope = np.polyfit(freqs, psd, 1)[0]
    features["spectral_slope"] = slope

    features["spectral_decrease"] = np.mean(
        (psd[1:] - psd[0]) / (freqs[1:] + 1e-8)
    )

    return features


In [4]:
features_list = []

audio_files = [
    f for f in os.listdir(RAW_AUDIO_DIR)
    if f.lower().endswith((".wav", ".mp3"))
]

print("Audio files found:", audio_files)

for file in audio_files:
    path = os.path.join(RAW_AUDIO_DIR, file)
    feats = extract_audio_features(path, file)
    features_list.append(feats)

df = pd.DataFrame(features_list)
df.head()


Audio files found: ['C01.mp3', 'C02.mp3', 'C03.mp3', 'C04.mp3', 'C05.mp3', 'C06.mp3', 'C07.mp3', 'C08.mp3', 'C09.mp3', 'C10.mp3']


,filename,recording_length,rms_power,zero_crossing_rate,crest_factor,mfcc_1_mean,mfcc_1_std,mfcc_2_mean,mfcc_2_std,mfcc_3_mean,...,mfcc_13_std,spectral_centroid,spectral_rolloff,spectral_spread,spectral_flatness,spectral_skewness,spectral_kurtosis,spectral_std,spectral_slope,spectral_decrease
0,C01.mp3,10.344490,0.038089,0.135228,14.321193,-241.303955,91.666275,84.142677,21.002813,-4.462087,...,6.532364,2704.818597,6107.294385,2786.489230,0.051419,16.852112,348.250119,21.021013,-0.001677,0.006595
1,C02.mp3,8.646531,0.025775,0.125209,16.920517,-327.702301,119.825050,91.567764,26.206709,-8.425058,...,6.866963,2444.247799,5257.334429,2619.219154,0.034675,8.877528,101.385738,6.324075,-0.000770,0.004322
2,C03.mp3,13.818776,0.063215,0.094785,9.508083,-213.037521,89.191406,116.747574,28.595161,-19.327377,...,7.169479,1897.756926,3703.765132,2121.751823,0.021194,5.990745,40.733205,31.311411,-0.003855,0.017557
3,C04.mp3,10.475102,0.093060,0.081651,7.570521,-213.189240,118.971703,124.040565,31.689281,-15.035368,...,6.932873,1749.728188,3425.756350,2105.044723,0.011045,5.914286,38.410436,64.781082,-0.007476,0.020954
4,C05.mp3,10.135510,0.089877,0.078488,8.532025,-199.172394,93.771118,126.659698,25.368902,-24.815289,...,8.760152,1683.891182,3190.338679,2068.883210,0.006417,5.443716,31.609686,38.163662,-0.004678,0.021986


In [5]:
output_csv = os.path.join(PROCESSED_AUDIO_DIR, "extracted_features.csv")
df.to_csv(output_csv, index=False)

print("Saved to:", output_csv)


Saved to: c:\Users\user\OneDrive\Desktop\portfolio\portfolio_projects\DRI_SWAASA-main\data\processed\audio\extracted_features.csv
